# --- Day 20: Particle Swarm ---
Suddenly, the GPU contacts you, asking for help. Someone has asked it to simulate too many particles, and it won't be able to finish them all in time to render the next frame at this rate.

It transmits to you a buffer (your puzzle input) listing each particle in order (starting with particle 0, then particle 1, particle 2, and so on). For each particle, it provides the X, Y, and Z coordinates for the particle's position (p), velocity (v), and acceleration (a), each in the format <X,Y,Z>.

Each tick, all particles are updated simultaneously. A particle's properties are updated in the following order:

- Increase the X velocity by the X acceleration.
- Increase the Y velocity by the Y acceleration.
- Increase the Z velocity by the Z acceleration.
- Increase the X position by the X velocity.
- Increase the Y position by the Y velocity.
- Increase the Z position by the Z velocity.

Because of seemingly tenuous rationale involving z-buffering, the GPU would like to know which particle will stay closest to position <0,0,0> in the long term. Measure this using the Manhattan distance, which in this situation is simply the sum of the absolute values of a particle's X, Y, and Z position.

For example, suppose you are only given two particles, both of which stay entirely on the X-axis (for simplicity). Drawing the current states of particles 0 and 1 (in that order) with an adjacent a number line and diagram of current X positions (marked in parenthesis), the following would take place:

```
p=< 3,0,0>, v=< 2,0,0>, a=<-1,0,0>    -4 -3 -2 -1  0  1  2  3  4
p=< 4,0,0>, v=< 0,0,0>, a=<-2,0,0>                         (0)(1)

p=< 4,0,0>, v=< 1,0,0>, a=<-1,0,0>    -4 -3 -2 -1  0  1  2  3  4
p=< 2,0,0>, v=<-2,0,0>, a=<-2,0,0>                      (1)   (0)

p=< 4,0,0>, v=< 0,0,0>, a=<-1,0,0>    -4 -3 -2 -1  0  1  2  3  4
p=<-2,0,0>, v=<-4,0,0>, a=<-2,0,0>          (1)               (0)

p=< 3,0,0>, v=<-1,0,0>, a=<-1,0,0>    -4 -3 -2 -1  0  1  2  3  4
p=<-8,0,0>, v=<-6,0,0>, a=<-2,0,0>                         (0)   
```

At this point, particle 1 will never be closer to <0,0,0> than particle 0, and so, in the long run, particle 0 will stay closest.

Which particle will stay closest to position `<0,0,0>` in the long term?

In [1]:
# the puzzle input
with open('puzzle_inputs/day20_input.txt') as f:
    data = f.read().strip().split("\n")
puzzle_input = [line for line in data]
puzzle_input[:3]

['p=<-833,-499,-1391>, v=<84,17,61>, a=<-4,1,1>',
 'p=<-168,3586,-2721>, v=<-61,-58,61>, a=<7,-13,8>',
 'p=<364,223,1877>, v=<31,-11,-71>, a=<-5,0,-3>']

This looks like a job for namedtuple and dicts:

In [110]:
from collections import namedtuple, defaultdict
import numpy as np

particle = namedtuple("Particle", ["pos", "vel", "acc"])
particles = {}

for i, line in enumerate(puzzle_input):
    p, v, a= line.split(", ")
    pos = np.array([int(i) for i in p.split("=<")[1][:-1].split(",")])
    vel = np.array([int(i) for i in v.split("=<")[1][:-1].split(",")])
    acc = np.array([int(i) for i in a.split("=<")[1][:-1].split(",")])
    particles[i] = particle(pos, vel, acc)

some helper functions:

In [4]:
def manhatten(list_nums):
    """takes in a list of numbers, returns manhatten distance"""
    return sum(abs(k) for k in list_nums)

## Part 1

For part 1, the particle with the smallest acc will stick around closest to the origin. in case of a tie, look at the velocities, the the position.

First up, the min acc is:

In [6]:
accels = [manhatten(p.acc) for k, p in particles.items()]
min_acc = min(accels)
f"Smallest acc: {min_acc}, Count {accels.count(min_acc)}"

'Smallest acc: 1, Count 2'

so the smallest acceleration is 1, with there being two particles. now to collect these two particles:

In [7]:
min_acc_particles = {k:p for k, p in particles.items() if manhatten(p.acc) == min_acc}
min_acc_particles

{21: Particle(pos=array([  348, -3515,  5362]), vel=array([   8,   97, -150]), acc=array([-1,  0,  0])),
 457: Particle(pos=array([-1271,   294,  5831]), vel=array([  37,  -25, -172]), acc=array([0, 1, 0]))}

In [8]:
for k, p in min_acc_particles.items():
    print(k, manhatten(p.pos), manhatten(p.vel))

21 9225 255
457 7396 234


The lower velocity particle, # `457` wins. If the velocity had tied, then consider the position.

# --- Part Two ---

To simplify the problem further, the GPU would like to remove any particles that collide. Particles collide if their positions ever exactly match. Because particles are updated simultaneously, more than two particles can collide at the same time and place. Once particles collide, they are removed and cannot collide with anything else after that tick.

For example:

```
p=<-6,0,0>, v=< 3,0,0>, a=< 0,0,0>    
p=<-4,0,0>, v=< 2,0,0>, a=< 0,0,0>    -6 -5 -4 -3 -2 -1  0  1  2  3
p=<-2,0,0>, v=< 1,0,0>, a=< 0,0,0>    (0)   (1)   (2)            (3)
p=< 3,0,0>, v=<-1,0,0>, a=< 0,0,0>

p=<-3,0,0>, v=< 3,0,0>, a=< 0,0,0>    
p=<-2,0,0>, v=< 2,0,0>, a=< 0,0,0>    -6 -5 -4 -3 -2 -1  0  1  2  3
p=<-1,0,0>, v=< 1,0,0>, a=< 0,0,0>             (0)(1)(2)      (3)   
p=< 2,0,0>, v=<-1,0,0>, a=< 0,0,0>

p=< 0,0,0>, v=< 3,0,0>, a=< 0,0,0>    
p=< 0,0,0>, v=< 2,0,0>, a=< 0,0,0>    -6 -5 -4 -3 -2 -1  0  1  2  3
p=< 0,0,0>, v=< 1,0,0>, a=< 0,0,0>                       X (3)      
p=< 1,0,0>, v=<-1,0,0>, a=< 0,0,0>

------destroyed by collision------    
------destroyed by collision------    -6 -5 -4 -3 -2 -1  0  1  2  3
------destroyed by collision------                      (3)         
p=< 0,0,0>, v=<-1,0,0>, a=< 0,0,0>
```

In this example, particles 0, 1, and 2 are simultaneously destroyed at the time and place marked X. On the next tick, particle 3 passes through unharmed.

**How many particles are left after all collisions are resolved?**

First up, del particles which occupy the same position. Here I compare every particle's position with every other one, adding the indexes to a set. It's a slow method so consider np.unique for a speedup:

In [115]:
def remove_collisions(particles=particles, verbose=False):
    """takes in a dictionary of particles and deletes particles in the same position"""
    
    collisions = set()
    
    for i, p in particles.items():
        for j, p2 in particles.items():
            if i == j: continue # don't compare particle to itself
            if np.all(p.pos == p2.pos):
                collisions.add(i)
                collisions.add(j)
                
    if verbose and len(collisions) > 0:
        print(f"found {len(collisions)} collisions")
        print(collisions)
        print("Deleting collisions")
    
    for key in collisions:
        try:
            del particles[key]
        except KeyError:
            pass
    
remove_collisions(verbose=True)

Now we need a way to update the particles position, one or multiple ticks at a time:

In [116]:
def update_particles(particles=particles, t=1):
    """updates particles by given timestep, defaulting to 1 ts"""
    for _ in range(t):
        for key, p in particles.items():
            vel = p.vel + p.acc
            pos = p.pos + vel
            acc = p.acc
            particles[key] = particle(pos, vel, acc)

update_particles()
particles[0]

Particle(pos=array([ -753,  -481, -1329]), vel=array([80, 18, 62]), acc=array([-4,  1,  1]))

In [119]:
# making sure the particles dictionary is fresh:
for i, line in enumerate(puzzle_input):
    p, v, a= line.split(", ")
    pos = np.array([int(i) for i in p.split("=<")[1][:-1].split(",")])
    vel = np.array([int(i) for i in v.split("=<")[1][:-1].split(",")])
    acc = np.array([int(i) for i in a.split("=<")[1][:-1].split(",")])
    particles[i] = particle(pos, vel, acc)

i = 0
while len(particles) > 1:
    remove_collisions(verbose=True)
    update_particles()
    i += 1
    if i % 20 == 0:
        print(f"Loop {i}, Partilce count: {len(particles)}")

found 21 collisions
{395, 396, 397, 398, 399, 400, 401, 402, 464, 465, 466, 467, 468, 469, 470, 89, 90, 91, 92, 93, 94}
Deleting collisions
found 6 collisions
{150, 151, 152, 153, 154, 155}
Deleting collisions
found 18 collisions
{384, 385, 448, 449, 446, 450, 451, 452, 453, 447, 376, 377, 378, 379, 380, 381, 382, 383}
Deleting collisions
found 23 collisions
{547, 548, 549, 550, 551, 41, 42, 43, 44, 45, 77, 78, 79, 80, 81, 82, 83, 84, 85, 369, 370, 371, 372}
Deleting collisions
found 11 collisions
{242, 243, 244, 245, 246, 403, 24, 25, 26, 27, 404}
Deleting collisions
found 15 collisions
{229, 230, 231, 232, 233, 234, 235, 236, 237, 295, 296, 297, 298, 299, 300}
Deleting collisions
found 32 collisions
{13, 14, 15, 16, 17, 18, 19, 533, 534, 31, 32, 33, 34, 35, 36, 37, 38, 39, 40, 198, 199, 200, 201, 202, 203, 490, 491, 492, 493, 494, 495, 496}
Deleting collisions
found 16 collisions
{544, 545, 546, 305, 306, 307, 308, 309, 310, 311, 312, 313, 314, 187, 188, 543}
Deleting collisions
foun

KeyboardInterrupt: 

In [121]:
len(particles)

448

'448' particles are left after all collisions are resolved.

## Notes:

- this is a slow way, instead of simulating I could math out which particles collide
- I got lucky, at only about loop 40 the particles stopped colliding, it could be much longer